In [80]:
"""
OBJETIVO: 
    - Extraer el titulo  y precios de los productos anunciados en la pagina de Fybeca Ecuador.
    - Aprender a realizar extracciones que requieran una accion de click para cargar datos.
    - Introducirnos a la logica de Selenium
    -Guardar los datos extraidos en un archivo excel
    -Guardar las imagenes de los productos en un carpeta especifica.
CREADO POR: Ivonne Bermeo
ULTIMA VEZ EDITADO: 2 Mayo 2024
"""

import requests 
import os 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
service = Service(executable_path="C:/Users/ivonn/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe")
options = webdriver.ChromeOptions()

driver = webdriver.Chrome(service=service, options=options)
try:
    driver.get("https://www.fybeca.com/")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    
    # Hacer clic en el 'Menú de Productos' para desplegar las opciones
    menu_productos = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//div[contains(text(),'Menú de Productos')]"))
    )
    menu_productos.click()

    time.sleep(2)

    # Hacer clic en 'Dermocosmética'
    dermocosmetica_link = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="dermocosmetica"]'))
    )
    dermocosmetica_link.click()

    time.sleep(2)  # Esperar a que el submenu se despliegue completamente

    # Hacer clic en 'Protección Solar'
    proteccion_solar_link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="proteccion_solar"]')) 
    )
    proteccion_solar_link.click()
    
    time.sleep(5) 
    
    lista_nombres= []
    lista_precios=[]
    direct_images=[]
    
    cantidad_items = len(driver.find_elements(By.XPATH, '//*[@id="product-search-results"]/div/div[2]/div[2]/div'))
    for i in range(1, cantidad_items ):
        # Extraer nombre del producto
        nom = driver.find_element(By.XPATH, f'//*[@id="product-search-results"]/div/div[2]/div[2]/div[{i}]/div/div/div[3]/div[1]/a')
        lista_nombres.append(nom.text)

        # Intentar extraer el precio principal
        precio = driver.find_element(By.XPATH, f'//*[@id="product-search-results"]/div/div[2]/div[2]/div[{i}]/div/div/div[3]/div[2]/span/span[1]/div/span').text
        if not precio:  # Si no hay texto en el precio principal, buscar el alternativo
            precio = driver.find_element(By.XPATH, f'//*[@id="product-search-results"]/div/div[2]/div[2]/div[{i}]/div/div/div[3]/div[2]/span/span[2]/span/span[2]/span/div/span').text
        
        lista_precios.append(precio)
        
        bimg=driver.find_element(By.XPATH,f'//*[@id="product-search-results"]/div/div[2]/div[2]/div[{i}]/div/div/div[2]/a/img')
        direct_images.append(bimg.get_attribute("src"))
        
        
    df = pd.DataFrame({
    "Nombres": lista_nombres,
    "Precios": lista_precios,
    "URL imagen":direct_images})    
    # Guardar el DataFrame en un archivo Excel
    df.to_excel("C:/Users/ivonn/Desktop/gem/prueba_gem.xlsx", index=False, engine='openpyxl')
    
    for i in range(len(df["URL imagen"])):
        res = requests.get(df["URL imagen"][i])
        nom_img = lista_nombres[i] + ".jpg"
        print(df["URL imagen"][i])
        path_to_save = os.path.join('C:/Users/ivonn/Pictures', nom_img)  # Construir el path completo donde se guardará la imagen
        with open(path_to_save, "wb") as img:
            img.write(res.content)

finally:
    time.sleep(2)
    driver.quit()
    
print(df)

https://www.fybeca.com/dw/image/v2/BDPM_PRD/on/demandware.static/-/Sites-masterCatalog_FybecaEcuador/default/dw5b95b973/images/large/100237037_1.jpg?sw=295&sh=295
https://www.fybeca.com/dw/image/v2/BDPM_PRD/on/demandware.static/-/Sites-masterCatalog_FybecaEcuador/default/dwa5075fea/images/large/100239643-PROTECTOR-SUNCARE-KIDS-150-G-FRASCO.jpg?sw=295&sh=295
https://www.fybeca.com/dw/image/v2/BDPM_PRD/on/demandware.static/-/Sites-masterCatalog_FybecaEcuador/default/dw75505a21/images/large/100109042-CREMA-ANTIAGE-SUNCARE-60-G-TUBO.JPG?sw=295&sh=295
https://www.fybeca.com/dw/image/v2/BDPM_PRD/on/demandware.static/-/Sites-masterCatalog_FybecaEcuador/default/dw4086b9d2/images/large/295996_1.jpg?sw=295&sh=295
https://www.fybeca.com/dw/image/v2/BDPM_PRD/on/demandware.static/-/Sites-masterCatalog_FybecaEcuador/default/dw30e51682/images/large/100115634_1.jpg?sw=295&sh=295
https://www.fybeca.com/dw/image/v2/BDPM_PRD/on/demandware.static/-/Sites-masterCatalog_FybecaEcuador/default/dw6643f155/imag